# 噪聲處理與降噪技術

## 學習目標
- 理解影像噪聲的類型與成因
- 掌握噪聲檢測與識別方法
- 學習各種降噪演算法
- 比較不同降噪技術的效果

**難度等級**: ⭐⭐⭐ (中級)  
**預估時間**: 90 分鐘  
**WBS編號**: 2.3.2

In [ ]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils.image_utils import load_image, resize_image
from utils.visualization import display_image, display_multiple_images
from utils.performance import time_function, benchmark_function

print(f'✅ OpenCV version: {cv2.__version__}')
print(f'✅ NumPy version: {np.__version__}')

## 1. 影像噪聲類型

### 常見噪聲分類

| 噪聲類型 | 特徵 | 成因 | 視覺效果 |
|---------|------|------|----------|
| **高斯噪聲** | 符合常態分佈 | 感測器熱噪聲、電子電路 | 整體模糊 |
| **椒鹽噪聲** | 隨機黑白點 | 傳輸錯誤、壞點 | 黑白斑點 |
| **斑點噪聲** | 乘性噪聲 | 雷達、超音波成像 | 顆粒狀 |
| **週期性噪聲** | 規律模式 | 電源干擾、摩爾紋 | 條紋或網格 |

### 噪聲數學模型

```
高斯噪聲:  g(x,y) = f(x,y) + N(μ, σ²)
椒鹽噪聲:  g(x,y) = {0 (黑), 255 (白), f(x,y) (原值)}
斑點噪聲:  g(x,y) = f(x,y) × (1 + N(0, σ²))
```

### 1.1 噪聲生成函數

In [ ]:
def add_gaussian_noise(image, mean=0, std=25):
    """
    Add Gaussian noise to image
    
    Args:
        image: Input image (uint8)
        mean: Mean of Gaussian distribution
        std: Standard deviation
    
    Returns:
        Noisy image
    """
    noise = np.random.normal(mean, std, image.shape)
    noisy = image.astype(np.float32) + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def add_salt_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):
    """
    Add salt and pepper noise
    
    Args:
        image: Input image
        salt_prob: Probability of salt (white) noise
        pepper_prob: Probability of pepper (black) noise
    
    Returns:
        Noisy image
    """
    noisy = image.copy()
    
    # Salt noise (white)
    salt_mask = np.random.random(image.shape[:2]) < salt_prob
    noisy[salt_mask] = 255
    
    # Pepper noise (black)
    pepper_mask = np.random.random(image.shape[:2]) < pepper_prob
    noisy[pepper_mask] = 0
    
    return noisy

def add_speckle_noise(image, std=0.1):
    """
    Add speckle (multiplicative) noise
    
    Args:
        image: Input image
        std: Standard deviation of noise
    
    Returns:
        Noisy image
    """
    noise = np.random.randn(*image.shape) * std
    noisy = image.astype(np.float32) * (1 + noise)
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

print('✅ Noise generation functions defined')

### 1.2 噪聲類型示範

In [ ]:
# Create test image
test_img = np.ones((300, 300), dtype=np.uint8) * 128
cv2.rectangle(test_img, (50, 50), (250, 250), 200, -1)
cv2.circle(test_img, (150, 150), 60, 80, -1)

# Generate different noise types
gaussian_noisy = add_gaussian_noise(test_img, mean=0, std=30)
salt_pepper_noisy = add_salt_pepper_noise(test_img, 0.02, 0.02)
speckle_noisy = add_speckle_noise(test_img, std=0.15)

# Display
images = [test_img, gaussian_noisy, salt_pepper_noisy, speckle_noisy]
titles = ['Original', 'Gaussian Noise\n(σ=30)', 'Salt & Pepper\n(p=0.02)', 'Speckle Noise\n(σ=0.15)']

display_multiple_images(images, titles, rows=1, cols=4, figsize=(16, 4), cmap='gray')

# Print noise statistics
print('\n噪聲統計分析:')
print('='*60)
for name, img in zip(titles[1:], images[1:]):
    diff = img.astype(np.float32) - test_img.astype(np.float32)
    print(f'{name:20} | Mean diff: {diff.mean():6.2f} | Std: {diff.std():6.2f}')

## 2. 噪聲檢測與識別

### 2.1 統計特徵分析

In [ ]:
def analyze_noise(image, original=None):
    """
    Analyze noise characteristics
    """
    print(f'\n影像統計分析:')
    print('='*60)
    print(f'Mean:   {image.mean():.2f}')
    print(f'Std:    {image.std():.2f}')
    print(f'Min:    {image.min()}')
    print(f'Max:    {image.max()}')
    
    if original is not None:
        # Calculate SNR (Signal-to-Noise Ratio)
        signal = original.astype(np.float32)
        noise = image.astype(np.float32) - signal
        
        signal_power = np.mean(signal ** 2)
        noise_power = np.mean(noise ** 2)
        
        if noise_power > 0:
            snr = 10 * np.log10(signal_power / noise_power)
            print(f'\nSNR:    {snr:.2f} dB')
        
        # Calculate PSNR
        mse = np.mean((signal - image.astype(np.float32)) ** 2)
        if mse > 0:
            psnr = 20 * np.log10(255.0 / np.sqrt(mse))
            print(f'PSNR:   {psnr:.2f} dB')
    
    # Histogram analysis
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    peak_value = np.argmax(hist)
    print(f'\nHistogram peak: {peak_value}')

# Analyze different noise types
print('\n【高斯噪聲分析】')
analyze_noise(gaussian_noisy, test_img)

print('\n\n【椒鹽噪聲分析】')
analyze_noise(salt_pepper_noisy, test_img)

print('\n\n【斑點噪聲分析】')
analyze_noise(speckle_noisy, test_img)

## 3. 空間域降噪方法

### 3.1 均值濾波 (Mean Filter)

In [ ]:
# Apply mean filter
mean_filtered = cv2.blur(gaussian_noisy, (5, 5))

# Display
display_multiple_images(
    [test_img, gaussian_noisy, mean_filtered],
    ['Original', 'Noisy (Gaussian)', 'Mean Filter 5x5'],
    rows=1, cols=3, figsize=(15, 5), cmap='gray'
)

# Calculate improvement
mse_noisy = np.mean((test_img.astype(float) - gaussian_noisy.astype(float))**2)
mse_filtered = np.mean((test_img.astype(float) - mean_filtered.astype(float))**2)

print(f'MSE (noisy):    {mse_noisy:.2f}')
print(f'MSE (filtered): {mse_filtered:.2f}')
print(f'Improvement:    {(1 - mse_filtered/mse_noisy)*100:.1f}%')

### 3.2 高斯濾波 (Gaussian Filter)

高斯濾波對高斯噪聲特別有效，同時能較好地保留邊緣。

In [ ]:
# Apply Gaussian filters with different kernel sizes
gaussian_3x3 = cv2.GaussianBlur(gaussian_noisy, (3, 3), 0)
gaussian_5x5 = cv2.GaussianBlur(gaussian_noisy, (5, 5), 0)
gaussian_7x7 = cv2.GaussianBlur(gaussian_noisy, (7, 7), 0)

# Display
images = [gaussian_noisy, gaussian_3x3, gaussian_5x5, gaussian_7x7]
titles = ['Noisy', 'Gaussian 3x3', 'Gaussian 5x5', 'Gaussian 7x7']

display_multiple_images(images, titles, rows=1, cols=4, figsize=(16, 4), cmap='gray')

# Compare quality
print('\n高斯濾波器比較 (PSNR):')
print('='*60)
for title, img in zip(titles, images):
    mse = np.mean((test_img.astype(float) - img.astype(float))**2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    print(f'{title:15} | PSNR: {psnr:6.2f} dB')

### 3.3 中值濾波 (Median Filter)

**最適合處理椒鹽噪聲**，能有效移除脈衝噪聲同時保留邊緣。

In [ ]:
# Apply median filter to salt & pepper noise
median_3x3 = cv2.medianBlur(salt_pepper_noisy, 3)
median_5x5 = cv2.medianBlur(salt_pepper_noisy, 5)
median_7x7 = cv2.medianBlur(salt_pepper_noisy, 7)

# Compare with Gaussian filter
gaussian_sp = cv2.GaussianBlur(salt_pepper_noisy, (5, 5), 0)

# Display
images = [salt_pepper_noisy, median_3x3, median_5x5, gaussian_sp]
titles = ['Noisy (S&P)', 'Median 3x3', 'Median 5x5', 'Gaussian 5x5']

display_multiple_images(images, titles, rows=1, cols=4, figsize=(16, 4), cmap='gray')

# Quality comparison
print('\n椒鹽噪聲降噪效果比較:')
print('='*60)
for title, img in zip(titles[1:], images[1:]):
    mse = np.mean((test_img.astype(float) - img.astype(float))**2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    print(f'{title:15} | PSNR: {psnr:6.2f} dB')

print('\n💡 觀察: 中值濾波對椒鹽噪聲效果遠優於高斯濾波！')

### 3.4 雙邊濾波 (Bilateral Filter)

**保邊降噪濾波器** - 在降噪的同時能很好地保留邊緣細節。

#### 雙邊濾波器特點
- 同時考慮**空間距離**和**像素值差異**
- 邊緣區域不會被過度平滑
- 適合需要保留細節的場景

In [ ]:
# Apply bilateral filter
bilateral = cv2.bilateralFilter(gaussian_noisy, d=9, sigmaColor=75, sigmaSpace=75)

# Compare with Gaussian filter
gaussian_compare = cv2.GaussianBlur(gaussian_noisy, (9, 9), 0)

# Display
images = [test_img, gaussian_noisy, bilateral, gaussian_compare]
titles = ['Original', 'Noisy', 'Bilateral Filter', 'Gaussian Filter']

display_multiple_images(images, titles, rows=1, cols=4, figsize=(16, 4), cmap='gray')

# Edge preservation comparison
def calculate_edge_preservation(original, denoised, noisy):
    """Calculate edge preservation index"""
    edges_orig = cv2.Canny(original, 50, 150)
    edges_denoised = cv2.Canny(denoised, 50, 150)
    
    preserved = np.sum(edges_orig & edges_denoised)
    total = np.sum(edges_orig)
    
    return (preserved / total * 100) if total > 0 else 0

bilateral_ep = calculate_edge_preservation(test_img, bilateral, gaussian_noisy)
gaussian_ep = calculate_edge_preservation(test_img, gaussian_compare, gaussian_noisy)

print('\n邊緣保留能力比較:')
print('='*60)
print(f'Bilateral Filter:  {bilateral_ep:.1f}% edges preserved')
print(f'Gaussian Filter:   {gaussian_ep:.1f}% edges preserved')
print(f'\n💡 雙邊濾波器保留了更多邊緣細節！')

## 4. 進階降噪技術

### 4.1 非局部均值去噪 (Non-Local Means Denoising)

**NLM** 是強大的降噪演算法，基於影像中相似區塊的統計。

#### 核心原理
```
1. 搜尋影像中相似的區塊 (patches)
2. 對相似區塊進行加權平均
3. 權重由區塊相似度決定
```

In [ ]:
# Load a real image for better demonstration
img_path = '../assets/images/basic/lenaColor.png'

if os.path.exists(img_path):
    real_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if real_img is not None:
        real_img = cv2.resize(real_img, (300, 300))
else:
    # Use synthetic image if real image not available
    real_img = test_img.copy()

# Add Gaussian noise
noisy_real = add_gaussian_noise(real_img, mean=0, std=25)

# Apply different denoising methods
gaussian_real = cv2.GaussianBlur(noisy_real, (5, 5), 0)
bilateral_real = cv2.bilateralFilter(noisy_real, 9, 75, 75)
nlm_real = cv2.fastNlMeansDenoising(noisy_real, None, h=10, templateWindowSize=7, searchWindowSize=21)

# Display comparison
images = [real_img, noisy_real, gaussian_real, bilateral_real, nlm_real]
titles = ['Original', 'Noisy\n(σ=25)', 'Gaussian', 'Bilateral', 'NLM']

display_multiple_images(images, titles, rows=1, cols=5, figsize=(20, 4), cmap='gray')

# Quality metrics
print('\n降噪方法效果比較 (PSNR):')
print('='*60)
for title, img in zip(titles[1:], images[1:]):
    mse = np.mean((real_img.astype(float) - img.astype(float))**2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    print(f'{title:15} | PSNR: {psnr:6.2f} dB')

### 4.2 彩色影像降噪

In [ ]:
# Load color image
if os.path.exists(img_path):
    color_img = cv2.imread(img_path)
    if color_img is not None:
        color_img = cv2.resize(color_img, (300, 300))
else:
    # Create synthetic color image
    color_img = cv2.cvtColor(real_img, cv2.COLOR_GRAY2BGR)

# Add noise to color image
noisy_color = add_gaussian_noise(color_img, mean=0, std=20)

# Apply denoising to color image
denoised_color = cv2.fastNlMeansDenoisingColored(
    noisy_color, None, 
    h=10,           # Filter strength for luminance
    hColor=10,      # Filter strength for color
    templateWindowSize=7, 
    searchWindowSize=21
)

# Display
display_multiple_images(
    [color_img, noisy_color, denoised_color],
    ['Original', 'Noisy (σ=20)', 'NLM Denoised'],
    rows=1, cols=3, figsize=(15, 5)
)

print('✅ 彩色影像降噪完成')

### 4.3 形態學降噪

使用形態學操作移除小型噪聲區域。

In [ ]:
# Apply morphological operations to salt & pepper noise
kernel = np.ones((3, 3), np.uint8)

# Opening (removes small white noise)
opening = cv2.morphologyEx(salt_pepper_noisy, cv2.MORPH_OPEN, kernel)

# Closing (removes small black noise)
closing = cv2.morphologyEx(salt_pepper_noisy, cv2.MORPH_CLOSE, kernel)

# Opening followed by closing
morph_denoised = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

# Compare with median filter
median_compare = cv2.medianBlur(salt_pepper_noisy, 5)

# Display
images = [salt_pepper_noisy, opening, closing, morph_denoised, median_compare]
titles = ['Noisy', 'Opening', 'Closing', 'Open+Close', 'Median Filter']

display_multiple_images(images, titles, rows=1, cols=5, figsize=(20, 4), cmap='gray')

print('💡 形態學操作對結構化噪聲特別有效')

## 5. 降噪效能比較

In [ ]:
# Performance benchmark
test_size = (512, 512)
test_image = np.random.randint(0, 256, test_size, dtype=np.uint8)
noisy_test = add_gaussian_noise(test_image, std=25)

# Define denoising methods
methods = [
    ('Mean Filter', lambda: cv2.blur(noisy_test, (5, 5))),
    ('Gaussian Filter', lambda: cv2.GaussianBlur(noisy_test, (5, 5), 0)),
    ('Median Filter', lambda: cv2.medianBlur(noisy_test, 5)),
    ('Bilateral Filter', lambda: cv2.bilateralFilter(noisy_test, 9, 75, 75)),
    ('NLM Denoising', lambda: cv2.fastNlMeansDenoising(noisy_test, None, 10, 7, 21)),
]

print(f'\n效能測試 (512x512 影像):')
print('='*80)
print(f'{"Method":20} | {"Time (ms)":>12} | {"Speed":>15}')
print('-'*80)

for name, func in methods:
    stats = benchmark_function(func, iterations=5)
    avg_time = stats['average'] * 1000
    fps = 1.0 / stats['average'] if stats['average'] > 0 else 0
    
    print(f'{name:20} | {avg_time:10.2f} ms | {fps:10.1f} fps')

print('\n💡 觀察: 速度與品質的權衡')
print('   - 最快: Mean/Gaussian Filter')
print('   - 平衡: Median/Bilateral Filter')
print('   - 最佳品質: NLM (但最慢)')

## 6. 降噪方法選擇指南

### 決策樹

```
噪聲類型?
├─ 高斯噪聲
│   ├─ 需要保留邊緣? → Bilateral Filter / NLM
│   └─ 追求速度? → Gaussian Filter
│
├─ 椒鹽噪聲
│   ├─ 小範圍噪聲? → Median Filter (3x3 或 5x5)
│   └─ 大範圍噪聲? → Morphological Operations
│
├─ 斑點噪聲
│   └─ NLM Denoising / Bilateral Filter
│
└─ 混合噪聲
    └─ 多步驟處理: Median → Bilateral → NLM
```

### 方法總結表

| 方法 | 適用噪聲 | 速度 | 品質 | 邊緣保留 |
|------|---------|------|------|----------|
| **Mean Filter** | 高斯 | ⭐⭐⭐⭐⭐ | ⭐⭐ | ❌ |
| **Gaussian Filter** | 高斯 | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ❌ |
| **Median Filter** | 椒鹽 | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ✅ |
| **Bilateral Filter** | 高斯 | ⭐⭐⭐ | ⭐⭐⭐⭐ | ✅✅ |
| **NLM Denoising** | 所有類型 | ⭐ | ⭐⭐⭐⭐⭐ | ✅✅ |
| **Morphological** | 椒鹽 | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⚠️ |

## 7. 實作練習

### 練習 1: 混合噪聲處理

In [ ]:
# TODO: 處理同時包含高斯噪聲和椒鹽噪聲的影像

# Create mixed noise
mixed_noisy = add_gaussian_noise(test_img, std=20)
mixed_noisy = add_salt_pepper_noise(mixed_noisy, 0.01, 0.01)

# Step 1: Remove salt & pepper noise with median filter
step1 = cv2.medianBlur(mixed_noisy, 5)

# Step 2: Remove Gaussian noise with bilateral filter
step2 = cv2.bilateralFilter(step1, 9, 75, 75)

# Display
display_multiple_images(
    [test_img, mixed_noisy, step1, step2],
    ['Original', 'Mixed Noise', 'After Median', 'After Bilateral'],
    rows=1, cols=4, figsize=(16, 4), cmap='gray'
)

# Calculate improvement
mse_noisy = np.mean((test_img.astype(float) - mixed_noisy.astype(float))**2)
mse_step1 = np.mean((test_img.astype(float) - step1.astype(float))**2)
mse_step2 = np.mean((test_img.astype(float) - step2.astype(float))**2)

print(f'MSE (noisy):  {mse_noisy:.2f}')
print(f'MSE (step1):  {mse_step1:.2f} (改善 {(1-mse_step1/mse_noisy)*100:.1f}%)')
print(f'MSE (step2):  {mse_step2:.2f} (總改善 {(1-mse_step2/mse_noisy)*100:.1f}%)')

### 練習 2: 自適應降噪

In [ ]:
# TODO: 根據噪聲程度自動選擇濾波器參數

def adaptive_denoising(image, noise_level='auto'):
    """
    Adaptive denoising based on noise level estimation
    """
    # Estimate noise level if auto
    if noise_level == 'auto':
        # Use Laplacian variance as noise estimate
        laplacian = cv2.Laplacian(image, cv2.CV_64F)
        noise_std = laplacian.var()
        
        if noise_std < 100:
            level = 'low'
        elif noise_std < 500:
            level = 'medium'
        else:
            level = 'high'
    else:
        level = noise_level
    
    # Select filter based on noise level
    if level == 'low':
        # Light Gaussian blur
        denoised = cv2.GaussianBlur(image, (3, 3), 0)
        method = 'Gaussian 3x3'
    elif level == 'medium':
        # Bilateral filter
        denoised = cv2.bilateralFilter(image, 7, 50, 50)
        method = 'Bilateral (d=7)'
    else:  # high
        # NLM denoising
        denoised = cv2.fastNlMeansDenoising(image, None, 15, 7, 21)
        method = 'NLM (h=15)'
    
    return denoised, level, method

# Test with different noise levels
low_noise = add_gaussian_noise(test_img, std=10)
med_noise = add_gaussian_noise(test_img, std=25)
high_noise = add_gaussian_noise(test_img, std=40)

results = []
titles = []

for name, img in [('Low', low_noise), ('Medium', med_noise), ('High', high_noise)]:
    denoised, level, method = adaptive_denoising(img)
    results.extend([img, denoised])
    titles.extend([f'{name} Noise', f'Auto: {method}'])
    print(f'{name} noise → Detected: {level} → Applied: {method}')

display_multiple_images(results, titles, rows=3, cols=2, figsize=(10, 12), cmap='gray')

## 8. 總結

### 重點回顧

1. **噪聲類型**
   - 高斯噪聲: 符合常態分佈
   - 椒鹽噪聲: 隨機黑白點
   - 斑點噪聲: 乘性噪聲

2. **降噪方法**
   - 空間域: Mean, Gaussian, Median, Bilateral
   - 進階方法: NLM, 形態學
   - 自適應降噪

3. **選擇原則**
   - 椒鹽噪聲 → Median Filter
   - 高斯噪聲 + 保邊 → Bilateral / NLM
   - 追求速度 → Gaussian Filter
   - 最佳品質 → NLM Denoising

4. **品質評估**
   - PSNR (Peak Signal-to-Noise Ratio)
   - MSE (Mean Squared Error)
   - 邊緣保留能力

### 下一步學習

- [x] 濾波與平滑 ✅
- [x] 形態學操作 ✅
- [x] 邊緣檢測 ✅
- [x] 直方圖處理 ✅
- [x] 噪聲處理 ✅ ← **你在這裡**
- [ ] 特徵檢測
- [ ] 物體識別

### 參考資源

- OpenCV Denoising: https://docs.opencv.org/4.x/d5/d69/tutorial_py_non_local_means.html
- 本專案 utils 模組: `utils/image_utils.py`
- 效能測試: `utils/performance.py`